- 전처리 어떻게 할까?

[ARTPRICE]
--> Title : 유지
--> Author
1. 순서대로 1,2,3,...,29,30 이렇게 붙이면?
--> year
1. 결측치 있는 것은 어떻게?
--> standard(그림크기)
1. 가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
2. 결측치 있는 것은 drop?
--> genre
1. 회화 / 판화 / 서예 / 서예-현판 / 입체 / 공예 / 사진-미디어 / 기타 로 구분하여 각각에 0,1,2,3... 식의 값을 부여
--> technique
1. 요소들이 'AAAA, BBBB, CCCC' 형태로 되어 있는 것들이 많아서 이것들을 다 분리해야 함
2. 분리한 이후 각 요소들을 포함하는지의 여부를 one-hot encoding으로 정리
3. 결측치 있는 것은 어떻게?
--> source
1. **옥션, **옥션 온라인으로 구분됨

1-1. 각각의 source를 그냥 0,1,2...로 값을 준다.

1-2. 온라인 여부를 따로 column을 만들고, K옥션 온라인과 K옥션을 통합하여 전처리한다.

--> price
1. 원화 표시, 콤마 표시 제거

[Economic]
- 주가, 비트코인, 유가
: 거래 당일날의 가격에 해당하는 종가 가격을 찾아 join
- 부동산 가격
: 거래 당월에 해당하는 가격상승 값을 찾아 join

[Correlation]
- 각 요소들과 price와의 상관관계를 linechart, heatmap 등으로 표현하여 시각화


In [186]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [187]:
df = pd.read_excel('./crawling_files/artprice_total.xlsx', engine='openpyxl')
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,김환기,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,김환기,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,김환기,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,김환기,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,김환기,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


In [188]:
df.isnull().sum()

title           0
author          0
year         1607
standard        3
genre           0
technique      89
source          0
date            0
price           0
dtype: int64

# ARTPRICE

## Author

김환기 : 0, 
이우환 : 1, 
박서보 : 2, 
정상화 : 3, 
김창열 : 4, 
박수근 : 5, 
천경자 : 6, 
이중섭 : 7, 
윤형근 : 8, 
이대원 : 9, 
유영국 : 10, 
장욱진 : 11, 
김종학 : 12, 
백남준 : 13, 
오치균 : 14, 
하종현 : 15, 
도상봉 : 16, 
이강소 : 17, 
이왈종 : 18, 
정선 : 19, 
김홍도 : 20, 
김태호 : 21, 
이상범 : 22, 
남관 : 23, 
이응노 : 24, 
전광영 : 25, 
김정희 : 26, 
서세옥 : 27, 
이성자 : 28, 
김기창 : 29

In [189]:
df['author'] = df['author'].map({'김환기' : 0, '이우환' : 1, '박서보' : 2, '정상화' : 3, '김창열' : 4, '박수근' : 5, '천경자' : 6, 
                                        '이중섭' : 7, '윤형근' : 8, '이대원' : 9, '유영국' : 10, '장욱진' : 11, '김종학' : 12, '백남준' : 13, 
                                        '오치균' : 14, '하종현' : 15, '도상봉' : 16, '이강소' : 17, '이왈종' : 18, '정선' : 19, '김홍도' : 20, 
                                        '김태호' : 21, '이상범' : 22, '남관' : 23, '이응노' : 24, '전광영' : 25, '김정희' : 26, '서세옥' : 27, 
                                        '이성자' : 28, '김기창' : 29})
df = df.astype({'author':'int'})

In [190]:
df.head()

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"


## Year

## Standard

가로*세로 곱한 값을 가져와서 '넓이'로 만든다.
결측치 있는 것은 drop?

In [191]:
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price
2980,Burunt Umber & Ultramarine Blue,8,NaN,NaN,회화,NaN,서울옥션 블루,2019/11/21,"￦90,000,000"
6168,하경산수,22,NaN,NaN,회화,종이에 수묵담채,마이아트옥션,2021/03/04,"￦16,000,000"
7013,고추,24,1979,NaN,회화,종이에 수묵담채,마이아트옥션,2017/05/22,"￦9,000,000"


결측치 데이터 3개 있어서 drop함

In [192]:
df.dropna(subset=['standard'], inplace=True)
df[df['standard'].isnull()]

,title,author,year,standard,genre,technique,source,date,price


데이터는 대개 다음과 같은 형태로 구성된다.

1. {숫자}x{숫자}cm : 대부분
2. {숫자}x{숫자}, {숫자}x{숫자}....
3. 지름 {숫자}cm
4. 지름 {숫자}x{숫자}

In [198]:
df = df[~df['standard'].str.contains('☓')]
df = df[~df['standard'].str.contains(':')]
df = df[~df['standard'].str.contains('지름')]
df = df[~df['standard'].str.contains('a.')]
df = df[~df['standard'].str.contains(',')]
df = df[~df['standard'].str.contains('외')]
df = df[~df['standard'].str.contains("\(")]
df = df[~df['standard'].str.contains("/")]
df = df[df['standard'].str.contains("×")]
df.reset_index(drop=True, inplace=True)
df

,title,author,year,standard,genre,technique,source,date,price
0,Duet,0,NaN,79.5×56.7cm,판화,"오프셋 프린트, 스크린 프린트(사후판화)",아트데이옥션 온라인,2021/06/30,"￦900,000"
1,27-VIII-70 #186,0,2015,100×72.7cm,판화,"피그먼트 프린트, 디아섹",서울옥션 온라인,2021/06/23,"￦850,000"
2,무제,0,1969,120.5×85.5cm,회화,코튼에 유채,K옥션,2021/06/23,"￦680,000,000"
3,27-XI-71 #211,0,1971,176.3×126.3cm,회화,면에 유채,서울옥션,2021/06/22,"￦3,050,000,000"
4,25-Ⅰ-68 Ⅳ,0,1968,58×37.4cm,회화,신문에 유채,서울옥션,2021/06/22,"￦72,000,000"
...,...,...,...,...,...,...,...,...,...
8177,독수리,29,1940년대,116×71cm,회화,종이에 수묵담채,아트데이옥션 온라인,2016/11/29,"￦7,000,000"
8178,군상,29,1940년대,14.8×21.8cm,회화,"종이에 먹, 연필",아트데이옥션 온라인,2016/11/29,"￦340,000"
8179,부엉이,29,1940년대,65×62.5cm,판화,석판화,K옥션 온라인,2016/11/29,"￦300,000"
8180,삽화,29,1940년대,13×17.5cm,회화,종이에 수묵,에이옥션 온라인,2016/11/23,"￦140,000"


In [200]:
df['area']=0
for j in range(len(df)):
    text = df['standard'][j]
    height = text.split('×')[0]
    width = text.split('×')[1]
    height = float(height)
    if 'cm' in width:
        width = float(width[:-2])
    else:
        width = float(width)
    df['area'][j] = height*width

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
